<h1 style='color:red'> Analise de ações com Python  <i style='color:black'>- Parte 1</i></h1>

<p> Ao decorrer deste Notebook, vou mostrar o processo que eu utilizo para realizar uma filtragem inicial nas ações, para visualizar as ações cujo indicadores estão melhores na minha visão 

<h2> Separaremos este notebook em Etapas </h2>

<li> Importação das Bibliotecas </li>
<li> Obtenção dos dados das ações </li>
<li> Tratamento dos dados </li>
<li> Visualização dos dados </li> 

<h2 style='color:red'> Importação das Bibliotecas </h2>

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import datetime 
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

<h2 style='color:red'> Obtenção dos dados das ações </h2>

<p> Para realizar a obtenção dos dados vamos utilizar de uma tecnica chamada de WebScrapping para realizar a raspagem de dados do site <b><a href='https://statusinvest.com.br/'> Status Invest </a></b> </p>

<h3> Criar as funções que serão utilizadas neste processo </h3>

In [3]:
# Realizar o GET para obter o HTML do site
def request(url):
    url_padrao = url
    
    response = requests.get(url)
    
    if response.status_code == 200:
        #print('Request Sucess!')
        html = response.content
        return BeautifulSoup(html,'html.parser')
    elif response.status_code == 429:
        print('HTTP 429 - Limite de requisições por tempo \n Esperando 1 minuto')
        time.sleep(60)
    elif response.status_code >= 500:
        print(f'HTTP {response.status_code} \n{response.content}')
    else:
        print('Verifique se o nome está correto. ')

In [4]:
# Obter a tag do HTML que obtem o nome da ação
def get_name(soup,acao):
    cabecalho = soup.find('h1',{'class':'lh-4'}).get_text().split(' - ')

    acao['nome'] = cabecalho[1]
    acao['acao'] = cabecalho[0]
    
    return acao

In [5]:
# Obter a tag do HTML que obtem o setor da ação
def get_setor(soup, acao):
    info_setor = soup.find('div',{'class':'card bg-main-gd-h white-text rounded ov-hidden pt-0 pb-0'})
    
    for info in info_setor:
        if info == '\n': continue 

        nomes =  info.find_all('span',{'class':'sub-value'})
        valores = info.find_all('strong',{'class':'value'})

    nomes = [nome.get_text() for nome in nomes]
    valores =[valor.get_text() for valor in valores]

    for i in range(len(nomes)):
        acao[nomes[i]] = valores[i]
    
    return acao

In [6]:
# Obter a tag do HTML que armazena os indicadores e informações das ações
def get_indicador(indicador,acao):
    
    for div in indicador:
        if div == '\n': continue 
            
        if div.find('h3',{'class':'title'}).get_text() == '':
            titulo = div.find_all('h3',{'class':'title'})[1].get_text()
        else:
            titulo = div.find('h3',{'class':'title'})
            if len(titulo) == 5:
                titulo = titulo.find('span',{'class':'d-inline-block mr-2'}).get_text()
            else: 
                titulo = div.find('h3',{'class':'title'}).get_text()

            
        valor = div.find('strong',{'class':'value'}).get_text().replace(',','.')
        acao[titulo] = valor
        
    return acao

<h3> Realizar o Web Scraping </h3>

<p> Primeiro precisamos obter o nome das ações onde vamos realizar o Scraping, para isso basta abrir a <b> <a href='https://statusinvest.com.br/acoes/busca-avancada'> Busca Avançada de Ações </a> </b> e filtrar por um setor e baixar seu arquivo .csv e renomear ele como empresas.csv.  

<b> <i> Obs: Poderiamos analisar diretamento o arquivo csv e não precisar utilizar o processo de Web Scraping , porém percebi que no arquivo CSV faltam alguns indicadores como <u> Liquidez Corrente </u>, <u> Dívida Líquida / EBITDA </u> e outros </b> </i>  

In [7]:
empresas = pd.read_csv('empresas.csv',sep=';')['TICKER']

In [8]:
empresas.head(2)

0    CSAN3
1    DMMO3
Name: TICKER, dtype: object

In [9]:
lista_acoes = []
before = datetime.datetime.now()

# percorrer pela ação obtendo seus indicadores
for empresa in empresas:
    url = 'https://statusinvest.com.br/'

    link = f'{url}acoes/{empresa}'
    soup = request(link)
    
    # Obter as informações das acoes
    acao = {}
    
    #Obter o nome e setor
    acao = get_name(soup,acao)
    acao = get_setor(soup,acao)
    
    
    # Obter o Header
    header = soup.find('div',{'class':'top-info has-special d-flex justify-between flex-wrap'})
    acao = get_indicador(header,acao)
    
    # Obter os números
    numeros = soup.find('div',{'class':'top-info info-3 sm d-flex justify-between mb-3'})
    acao = get_indicador(numeros,acao)
    
    # Obter os indicadores
    indicadores = soup.find_all('div',{'d-flex flex-wrap align-items-center justify-start'})[:5]
    
    for indicador in indicadores:
        acao = get_indicador(indicador,acao)
    
    #LINK
    acao['link'] = link
    
    lista_acoes.append(acao)
    
after = datetime.datetime.now()
print(f'Tempo de execução {after - before}')

Tempo de execução 0:00:15.213893


<h3> Criar o DataFrame com as informações </h3>

<p> Agora basta criarmos o DataFrame inicial no qual iremos trabalhar </p>

In [10]:
df_raw = pd.DataFrame(lista_acoes)
df = df_raw.copy()

<h2 style='color:red'> Tratamento dos dados </h2>

<p> Visualização do DataFrame Inicial </p>

In [11]:
df.head(2)

,nome,acao,Setor de Atuação,Subsetor de Atuação,Segmento de Atuação,Valor atual,Min. 52 semanas,Máx. 52 semanas,Dividend Yield,Valorização (12m),Patrimônio líquido,Ativos,Ativo circulante,Dívida bruta,Disponibilidade,Dívida líquida,Valor de mercado,Valor de firma,Nº total de papéis,Segmento de listagem,Free Float,,D.Y,P/L,PEG Ratio,P/VP,EV/EBITDA,EV/EBIT,P/EBITDA,P/EBIT,VPA,P/Ativo,LPA,P/SR,P/Cap. Giro,P/Ativo Circ. Liq.,Dív. líquida/PL,Dív. líquida/EBITDA,Dív. líquida/EBIT,PL/Ativos,Passivos/Ativos,Liq. corrente,M. Bruta,M. EBITDA,M. EBIT,M. Líquida,ROE,ROA,ROIC,Giro ativos,CAGR Receitas 5 anos,CAGR Lucros 5 anos,link
0,COSAN,CSAN3,Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,17.25,16.77,24.58,4.65,-25.07%,15.647.431.000,96.006.859.000,24.194.616.000,43.349.331.000,12.083.132.000,31.266.199.000,32.327.723.577,63.593.922.577,1.874.070.932,Novo Mercado,62.49%,,4.65%,6.56,0.07,2.07,5.40,7.06,2.74,3.59,8.35,0.34,2.63,1.04,2.84,-0.45,2.00,2.65,3.47,0.16,0.68,1.89,23.73%,37.88%,28.95%,15.84%,31.49%,5.13%,11.19%,0.32,26.99%,36.60%,https://statusinvest.com.br/acoes/CSAN3
1,DOMMO ENERGIA,DMMO3,Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,1.74,0.44,1.86,-,125.97%,-503.962.000,343.940.000,201.458.000,82.000,77.487.000,-77.405.000,886.852.020,809.447.020,509.685.069,,43.79%,,-%,-39.86,0.38,-1.76,-,27.61,-,30.25,-0.99,2.58,-0.04,3.01,8.63,-6.22,-,-,-2.64,-1.47,2.47,2.04,65.75%,-%,9.94%,-7.55%,-4.42%,-6.47%,-5.82%,0.86,-1.34%,-%,https://statusinvest.com.br/acoes/DMMO3


<h3> Filtrar as colunas </h3>

<p> Verificar as colunas e tipos de dados </p>

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   nome                  15 non-null     object
 1   acao                  15 non-null     object
 2   Setor de Atuação      15 non-null     object
 3   Subsetor de Atuação   15 non-null     object
 4   Segmento de Atuação   15 non-null     object
 5   Valor atual           15 non-null     object
 6   Min. 52 semanas       15 non-null     object
 7   Máx. 52 semanas       15 non-null     object
 8   Dividend Yield        15 non-null     object
 9   Valorização (12m)     15 non-null     object
 10  Patrimônio líquido    15 non-null     object
 11  Ativos                15 non-null     object
 12  Ativo circulante      15 non-null     object
 13  Dívida bruta          15 non-null     object
 14  Disponibilidade       15 non-null     object
 15  Dívida líquida        15 non-null     obje

<p> Com base nessas informações percebemos que os tipos de dados estão errados e existem diversas colunas que não seram utilizadas neste momento </p>

<p> Portanto vamos realizar uma filtragem de colunas inicialmente deixando somente as colunas que irie utilizar em minhas analises futuramente </p>

In [13]:
df.drop(columns=['nome','Segmento de Atuação','\xa0'],inplace = True,errors='ignore')
df = df[['acao','Subsetor de Atuação','Valor atual','Dividend Yield','Valorização (12m)','P/L','P/VP','EV/EBITDA','PEG Ratio','CAGR Receitas 5 anos','CAGR Lucros 5 anos','M. EBITDA','ROE','ROIC','Dívida líquida','Dív. líquida/PL','Dív. líquida/EBITDA','Liq. corrente','link']]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   acao                  15 non-null     object
 1   Subsetor de Atuação   15 non-null     object
 2   Valor atual           15 non-null     object
 3   Dividend Yield        15 non-null     object
 4   Valorização (12m)     15 non-null     object
 5   P/L                   15 non-null     object
 6   P/VP                  15 non-null     object
 7   EV/EBITDA             15 non-null     object
 8   PEG Ratio             15 non-null     object
 9   CAGR Receitas 5 anos  15 non-null     object
 10  CAGR Lucros 5 anos    15 non-null     object
 11  M. EBITDA             15 non-null     object
 12  ROE                   15 non-null     object
 13  ROIC                  15 non-null     object
 14  Dívida líquida        15 non-null     object
 15  Dív. líquida/PL       15 non-null     obje

<p> Agora já temos menos colunas, porém ainda precisamos ajustar seu tipo de dados para os tipos adequados de INT e FLOAT </p>

<h3> Ajustar os tipos de dados </h3>

In [15]:
df = df.replace(['-','-%','--%'],'0') 

<h4> Ajustar os registros de % </h4>

In [16]:
colunas_porcentagem = [coluna for coluna in df.columns if '%' in df[coluna].astype(str).str.slice(-1).unique()]

In [17]:
# Tratativa para as colunas de %
for coluna in colunas_porcentagem:
    df[coluna] = df[coluna].str.replace('%','')
    df[coluna] = df[coluna].apply(lambda x: x.replace('.','') if x[:-3].find('.') != -1 else x)
    df[coluna] = df[coluna].astype(float)
    
df.columns  = [coluna + ' %' if coluna in colunas_porcentagem else coluna for coluna in df.columns]

<h4> Ajustar os registros cujo valor seja FLOAT </h4>

In [18]:
colunas_float = ['Valor atual','Dividend Yield','P/L','P/VP','EV/EBITDA','PEG Ratio','Dívida líquida','Dív. líquida/PL','Dív. líquida/EBITDA','Liq. corrente']

In [19]:
for coluna in colunas_float:
    df[coluna] = df[coluna].apply(lambda valor : valor[:-3].replace('.','') + valor[-3:])
    if coluna == 'Dívida líquida':
        df[coluna] = df[coluna].astype(np.int64)
    df[coluna] = df[coluna].astype(float)

<p> Agora já temos o DataFrame com os dados devidamente corretos </p>

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   acao                    15 non-null     object 
 1   Subsetor de Atuação     15 non-null     object 
 2   Valor atual             15 non-null     float64
 3   Dividend Yield          15 non-null     float64
 4   Valorização (12m) %     15 non-null     float64
 5   P/L                     15 non-null     float64
 6   P/VP                    15 non-null     float64
 7   EV/EBITDA               15 non-null     float64
 8   PEG Ratio               15 non-null     float64
 9   CAGR Receitas 5 anos %  15 non-null     float64
 10  CAGR Lucros 5 anos %    15 non-null     float64
 11  M. EBITDA %             15 non-null     float64
 12  ROE %                   15 non-null     float64
 13  ROIC %                  15 non-null     float64
 14  Dívida líquida          15 non-null     floa

<p> Observe que agora temos um conjunto de dados mais organizados, facilitanto o entendimento do mesmo

In [21]:
df.head(2)

,acao,Subsetor de Atuação,Valor atual,Dividend Yield,Valorização (12m) %,P/L,P/VP,EV/EBITDA,PEG Ratio,CAGR Receitas 5 anos %,CAGR Lucros 5 anos %,M. EBITDA %,ROE %,ROIC %,Dívida líquida,Dív. líquida/PL,Dív. líquida/EBITDA,Liq. corrente,link
0,CSAN3,Petróleo. Gás e Biocombustíveis,17.25,4.65,-25.07,6.56,2.07,5.4,0.07,26.99,36.6,37.88,31.49,11.19,3.126620e+10,2.0,2.65,1.89,https://statusinvest.com.br/acoes/CSAN3
1,DMMO3,Petróleo. Gás e Biocombustíveis,1.74,0.00,125.97,-39.86,-1.76,0.0,0.38,-1.34,0.0,0.00,-4.42,-5.82,-7.740500e+07,0.0,0.00,2.04,https://statusinvest.com.br/acoes/DMMO3


# Visualização dos dados

<p> Para visualização, vou ajustar alguns pontos no proprio DataFrame para facilitar a visualização dos dados, para isso vou usar algumas cores para sinalizar seus indicadores, por exemplo:<br>
    <b style='color:green'>Verde: Bom</b> <br>
    <b style='color:red'>Vermelho: Ruim</b> <br>

<p> <b> <i> Lembrando que esse modo de analisar ações é o modo do qual eu utilizo, o que não significa que é o melhor ou que esteja 100% correto, mas sim o que eu particularmente gosto de usar para analisar as ações das quais pretendo comprar, sendo esta uma primeira filtragem das ações com base nos indicadores, após isso realizo uma analise mais aprofundada individualmente sobre as ações que escolher com base nesta filtragem.</i></b></p>

In [22]:
# Criação de funções para setar as alterações dependendo da cor 

def good_value():
    return 'background-color: #2E924C; font-weight: bold'

def medium_value():
    return 'background-color: #FFFFE5; color:black; opacity: 95%'

def bad_value():
    return 'background-color: red; font-weight: bold'

In [23]:
# Ordenar pelo setor
df = df.sort_values(by='Subsetor de Atuação')

In [24]:
# Remover uma ação cujo indicador estava muito alterao | com Margem de 271124.00%
df = df.drop(5)

In [25]:
df_view = (
df.style
            # Ajustar as casas decimais
            .format('{:.2f}', na_rep='MISS',subset=[coluna  for coluna in df.columns if coluna not in ('acao','Subsetor de Atuação','link')])
            .format('{:.2f}%', na_rep='MISS',subset=['Valorização (12m) %','M. EBITDA %','ROE %','ROIC %','CAGR Receitas 5 anos %','CAGR Lucros 5 anos %'])
            # Indicadores de Preço
            .applymap(lambda v: good_value() if (v > 0) and (v < 20)  else medium_value(), subset=['P/L'])
            .applymap(lambda v: good_value() if (v > 2)  else (medium_value() if v >= 0 and v <= 2 else bad_value()), subset=['P/VP'])
             #Indicadores de Crescimento
            .applymap(lambda v: good_value() if (v < 1) and (v > 0)  else (medium_value() if v >= 2 and v <= 2 else bad_value()),subset=['PEG Ratio'])
            # Rentabilidade
            .background_gradient(cmap='YlGn',subset=['M. EBITDA %','ROE %','ROIC %'])
            # Indicadores de Individamentos
            .applymap(lambda v: good_value() if (v < 1)  else (medium_value() if v >= 1 and v<= 3 else bad_value()), 
                      subset=['Dív. líquida/EBITDA'])
)

<h2> Visualizando as ações e seus indicadores </h2>

<p> Perceba que agora fica muito mais fácil visualizar as ações e seus indicadores, sabendo que por exemplos as ações em vermelho possuem indicadores não tão atrativos com base no tipo de ação que eu busco. Enquanto que as ações verdes estão com indicadores mais aceitaveis com base nos critérios que eu utilizo</p>

In [26]:
df_view

,acao,Subsetor de Atuação,Valor atual,Dividend Yield,Valorização (12m) %,P/L,P/VP,EV/EBITDA,PEG Ratio,CAGR Receitas 5 anos %,CAGR Lucros 5 anos %,M. EBITDA %,ROE %,ROIC %,Dívida líquida,Dív. líquida/PL,Dív. líquida/EBITDA,Liq. corrente,link
0,CSAN3,Petróleo. Gás e Biocombustíveis,17.25,4.65,-25.07%,6.56,2.07,5.40,0.07,26.99%,36.60%,37.88%,31.49%,11.19%,31266199000.00,2.00,2.65,1.89,https://statusinvest.com.br/acoes/CSAN3
1,DMMO3,Petróleo. Gás e Biocombustíveis,1.74,0.00,125.97%,-39.86,-1.76,0.00,0.38,-1.34%,0.00%,0.00%,-4.42%,-5.82%,-77405000.00,0.00,0.00,2.04,https://statusinvest.com.br/acoes/DMMO3
2,ENAT3,Petróleo. Gás e Biocombustíveis,14.10,12.12,6.66%,3.72,0.96,0.00,0.04,30.52%,45.79%,0.00%,25.67%,25.67%,-992146000.00,-0.25,0.00,2.63,https://statusinvest.com.br/acoes/ENAT3
3,LUPA3,Petróleo. Gás e Biocombustíveis,4.20,0.00,-19.85%,-13.19,0.98,-11.58,0.12,-7.55%,0.00%,-19.40%,-7.45%,-10.41%,124969000.00,1.00,-5.85,2.33,https://statusinvest.com.br/acoes/LUPA3
4,OGXP3,Petróleo. Gás e Biocombustíveis,1.63,0.00,0.00%,-2.15,-0.98,0.00,0.02,0.00%,0.00%,0.00%,-45.53%,10.57%,-2000.00,0.00,0.00,0.51,https://statusinvest.com.br/acoes/OGXP3
6,PETR3,Petróleo. Gás e Biocombustíveis,32.60,51.13,86.18%,2.63,1.04,1.66,0.05,9.88%,0.00%,62.05%,39.35%,30.90%,180369000000.00,0.44,0.51,1.31,https://statusinvest.com.br/acoes/PETR3
7,PETR4,Petróleo. Gás e Biocombustíveis,29.21,57.06,77.46%,2.36,0.93,1.66,0.04,9.88%,0.00%,62.05%,39.35%,30.90%,180369000000.00,0.44,0.51,1.31,https://statusinvest.com.br/acoes/PETR4
8,PRIO3,Petróleo. Gás e Biocombustíveis,27.87,0.00,21.97%,9.16,3.01,5.79,0.04,61.68%,61.88%,69.62%,32.87%,22.43%,86638000.00,0.01,0.02,8.20,https://statusinvest.com.br/acoes/PRIO3
9,RAIZ4,Petróleo. Gás e Biocombustíveis,4.16,1.59,-38.64%,14.85,1.93,3.30,0.41,0.00%,0.00%,9.15%,13.03%,10.57%,24465012000.00,1.10,1.20,1.16,https://statusinvest.com.br/acoes/RAIZ4
10,RECV3,Petróleo. Gás e Biocombustíveis,24.99,0.65,24.20%,11.64,2.27,0.00,0.02,0.00%,0.00%,0.00%,19.50%,15.52%,-1071498000.00,-0.33,0.00,1.29,https://statusinvest.com.br/acoes/RECV3


<p> Esta é uma maneira que eu utilizo para visualizar e filtrar ações, podemos também realizar diversas outras analises com base neste conjunto de dados, como verificar as distribuições, tendências e comportamentos dos indicadores para este conjunto de ações </p>

Mas isto irei realizar futuramente, onde desenvolverei mais um projeto o qual terá como foco a aplicação de tecnicas de <b> data visualization </b> como a criação de diversos gráficos. 

<h2 style='color:blue'> Espero que este conteúdo consiga lhe ajudar de alguma forma! </h2>